In [1]:
from IPython.display import clear_output
%pip install matplotlib
%pip install numpy
%pip install imageio
clear_output()

In [2]:
import os
import imageio
from matplotlib import rcParams
from timeit import default_timer
import numpy as np
from scipy import special as sp
from math import inf, sqrt, pi, log
from random import random, shuffle
from collections import deque
from heapq import heapify, heappop, heappush
from matplotlib import pyplot as plt

In [3]:
def is_sorted(data):
    return all([a <= b for a,b in zip(data, data[1:])])

def bogosort(data):
    inter = [list(data)]
    while not is_sorted(data):
        shuffle(data)
        inter.append(list(data))
    return data, inter


In [4]:
N = 6
data = [random() for i in range(N)]
_, inter = bogosort(data)
k = 0

for arr in inter:
    plt.figure(figsize=(4,4))
    plt.bar(range(N), arr)
    plt.title(f"{k}-th frame")

    plt.savefig(f'gifs/tmp/{k}.png')
    plt.clf()
    plt.close()
    k += 1

with imageio.get_writer('gifs/bogo.gif', mode='I') as writer:
    for filename in [f'gifs/tmp/{i}.png' for i in range(k)]:
        image = imageio.imread(filename)
        writer.append_data(image)
dir = 'gifs/tmp'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [5]:
def merge(A, B):
    result = []
    i, j = 0, 0
    while i < len(A) and j < len(B):
        if A[i] < B[j]:
            result.append(A[i])
            i += 1
        else:
            result.append(B[j])
            j += 1
    return result + A[i:] + B[j:]

def loopMergeSort(L):
    if len(L) < 1:
        return L
    Ls = [[x] for x in L]
    inter = [Ls]
    while len(Ls) > 1:
        Ls2 = []
        for i in range(0, len(Ls) - 1, 2):
            Ls2.append(merge(Ls[i], Ls[i + 1]))
        if len(Ls) %2 == 1:
            Ls2.append(Ls[-1])
        Ls = Ls2
        inter.append(Ls)
    return Ls[0], inter

In [6]:
N = 10000
data = [random() for i in range(N)]
_, inter = loopMergeSort(data)
k = 0

for arr in inter:
    plt.figure(figsize=(6,6))
    total = 0
    for sub in arr:
        plt.bar(range(total, total + len(sub)), sub)
        total += len(sub)
    plt.title(f"{k}-th frame")

    plt.savefig(f'gifs/tmp/{k}.png')
    plt.clf()
    plt.close()
    k += 1

with imageio.get_writer('gifs/merge.gif', mode='I') as writer:
    for filename in [f'gifs/tmp/{i}.png' for i in range(k)]:
        image = imageio.imread(filename)
        writer.append_data(image)
dir = 'gifs/tmp'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [7]:
class FibonacciHeap:

    # internal node class
    class Node:
        def __init__(self, key, value):
            self.key = key
            self.value = value
            self.parent = self.child = self.left = self.right = None
            self.degree = 0
            self.mark = False

    # function to iterate through a doubly linked list
    def iterate(self, head):
        node = stop = head
        flag = False
        while True:
            if node == stop and flag is True:
                break
            elif node == stop:
                flag = True
            yield node
            node = node.right

    # pointer to the head and minimum node in the root list
    root_list, min_node = None, None

    # maintain total node count in full fibonacci heap
    total_nodes = 0

    # return min node in O(1) time
    def find_min(self):
        return self.min_node

    # extract (delete) the min node from the heap in O(log n) time
    # amortized cost analysis can be found here (http://bit.ly/1ow1Clm)
    def extract_min(self):
        z = self.min_node
        if z is not None:
            if z.child is not None:
                # attach child nodes to root list
                children = [x for x in self.iterate(z.child)]
                for i in range(0, len(children)):
                    self.merge_with_root_list(children[i])
                    children[i].parent = None
            self.remove_from_root_list(z)
            # set new min node in heap
            if z == z.right:
                self.min_node = self.root_list = None
            else:
                self.min_node = z.right
                self.consolidate()
            self.total_nodes -= 1
        return z

    # insert new node into the unordered root list in O(1) time
    # returns the node so that it can be used for decrease_key later
    def insert(self, key, value=None):
        n = self.Node(key, value)
        n.left = n.right = n
        self.merge_with_root_list(n)
        if self.min_node is None or n.key < self.min_node.key:
            self.min_node = n
        self.total_nodes += 1
        return n

    # modify the key of some node in the heap in O(1) time
    def decrease_key(self, x, k):
        if k > x.key:
            return None
        x.key = k
        y = x.parent
        if y is not None and x.key < y.key:
            self.cut(x, y)
            self.cascading_cut(y)
        if x.key < self.min_node.key:
            self.min_node = x

    # merge two fibonacci heaps in O(1) time by concatenating the root lists
    # the root of the new root list becomes equal to the first list and the second
    # list is simply appended to the end (then the proper min node is determined)
    def merge(self, h2):
        H = FibonacciHeap()
        H.root_list, H.min_node = self.root_list, self.min_node
        # fix pointers when merging the two heaps
        last = h2.root_list.left
        h2.root_list.left = H.root_list.left
        H.root_list.left.right = h2.root_list
        H.root_list.left = last
        H.root_list.left.right = H.root_list
        # update min node if needed
        if h2.min_node.key < H.min_node.key:
            H.min_node = h2.min_node
        # update total nodes
        H.total_nodes = self.total_nodes + h2.total_nodes
        return H

    # if a child node becomes smaller than its parent node we
    # cut this child node off and bring it up to the root list
    def cut(self, x, y):
        self.remove_from_child_list(y, x)
        y.degree -= 1
        self.merge_with_root_list(x)
        x.parent = None
        x.mark = False

    # cascading cut of parent node to obtain good time bounds
    def cascading_cut(self, y):
        z = y.parent
        if z is not None:
            if y.mark is False:
                y.mark = True
            else:
                self.cut(y, z)
                self.cascading_cut(z)

    # combine root nodes of equal degree to consolidate the heap
    # by creating a list of unordered binomial trees
    def consolidate(self):
        A = [None] * int(log(self.total_nodes) * 2)
        nodes = [w for w in self.iterate(self.root_list)]
        for w in range(0, len(nodes)):
            x = nodes[w]
            d = x.degree
            while A[d] != None:
                y = A[d]
                if x.key > y.key:
                    temp = x
                    x, y = y, temp
                self.heap_link(y, x)
                A[d] = None
                d += 1
            A[d] = x
        # find new min node - no need to reconstruct new root list below
        # because root list was iteratively changing as we were moving
        # nodes around in the above loop
        for i in range(0, len(A)):
            if A[i] is not None:
                if A[i].key < self.min_node.key:
                    self.min_node = A[i]

    # actual linking of one node to another in the root list
    # while also updating the child linked list
    def heap_link(self, y, x):
        self.remove_from_root_list(y)
        y.left = y.right = y
        self.merge_with_child_list(x, y)
        x.degree += 1
        y.parent = x
        y.mark = False

    # merge a node with the doubly linked root list
    def merge_with_root_list(self, node):
        if self.root_list is None:
            self.root_list = node
        else:
            node.right = self.root_list.right
            node.left = self.root_list
            self.root_list.right.left = node
            self.root_list.right = node

    # merge a node with the doubly linked child list of a root node
    def merge_with_child_list(self, parent, node):
        if parent.child is None:
            parent.child = node
        else:
            node.right = parent.child.right
            node.left = parent.child
            parent.child.right.left = node
            parent.child.right = node

    # remove a node from the doubly linked root list
    def remove_from_root_list(self, node):
        if node == self.root_list:
            self.root_list = node.right
        node.left.right = node.right
        node.right.left = node.left

    # remove a node from the doubly linked child list
    def remove_from_child_list(self, parent, node):
        if parent.child == parent.child.right:
            parent.child = None
        elif parent.child == node:
            parent.child = node.right
            node.right.parent = parent
        node.left.right = node.right
        node.right.left = node.left

    def isEmpty(self):
        return self.root_list == None

In [8]:
N = 100
xs = np.random.random((N, 1))
ys = np.random.random((N, 1))

G = (xs - xs.T) ** 2.0 + (ys - ys.T) ** 2.0

def dijkstra(G, s, xs, ys):
    N = G.shape[0]
    d = np.ones(N) * inf
    d[s] = 0
    prevs = np.ones(N, dtype='int64') * -1
    prevs[s] = s
    Q = FibonacciHeap()
    fibnodes = []
    k = 0
    for v in range(N):
        node = Q.insert(d[v], v)
        fibnodes.append(node)
    while not Q.isEmpty():
        u = Q.extract_min().value
        for v in range(N):
            if d[v] > d[u] + G[u, v]:
                d[v] = d[u] + G[u, v]
                prevs[v] = u
                Q.decrease_key(fibnodes[v], d[v])
        plt.figure(figsize=(6,6))
        plt.scatter([xs[s, 0]], [ys[s, 0]], c='b')
        plt.scatter([xs[u, 0]], [ys[u, 0]], c='g')
        for i in range(N):
            x0, x1 = xs[i, 0], xs[prevs[i], 0]
            y0, y1 = ys[i, 0], ys[prevs[i], 0]
            plt.plot([x0, x1], [y0, y1], 'r')
        plt.title(f"{k}-th frame")
        plt.savefig(f'gifs/tmp/{k}.png')
        plt.clf()
        plt.close()
        k += 1
    with imageio.get_writer('gifs/dijkstra.gif', mode='I') as writer:
        for filename in [f'gifs/tmp/{i}.png' for i in range(k)]:
            image = imageio.imread(filename)
            writer.append_data(image)
    dir = 'gifs/tmp'
    for f in os.listdir(dir):
        os.remove(os.path.join(dir, f))

dijkstra(G, 0, xs, ys)

In [9]:
MST = []
N = 50
heads = {i: i for i in range(N)}
k = 0
ntrees = N

xs = np.random.random(N)
ys = np.random.random(N)

def distSquare(i, j):
    return (xs[i] - xs[j]) ** 2 + (ys[i] - ys[j]) ** 2

dists = [(i, j, distSquare(i, j)) for i in range(N) for j in range(N) if i < j]

dists.sort(key=(lambda x: x[2]))

def getHead(i):
    if heads[i] == i:
        return i
    else:
        heads[i] = getHead(heads[i])
        return heads[i]

for (i, j, _) in dists:
    headi, headj = getHead(i), getHead(j)
    plt.figure(figsize=(6,6))
    for (u, v) in MST:
        plt.plot([xs[u], xs[v]], [ys[u], ys[v]], 'r')
    if headi != headj:
        ntrees -= 1
        MST.append((i, j))
        heads[headi] = headj
        plt.plot([xs[i], xs[j]], [ys[i], ys[j]], 'y')
    else:
        plt.plot([xs[i], xs[j]], [ys[i], ys[j]], 'g')
    plt.xlim(0, 1)
    plt.ylim(0, 1)

    plt.savefig(f'gifs/tmp/{k}.png')
    plt.clf()
    plt.close()
    k += 1
    if ntrees < 2:
        break

with imageio.get_writer('gifs/MST.gif', mode='I') as writer:
    for filename in [f'gifs/tmp/{i}.png' for i in range(k)]:
        image = imageio.imread(filename)
        writer.append_data(image)
dir = 'gifs/tmp'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))